In [ ]:
youtube_search = ["youtube.com","youtu.be"]
pinterest_search = ['pinterest.com','pinimg']
reddit_search = ['reddit.com','redd.it']
image_search = ['img','png','jpeg','jpg']

In [1]:
# %run "/Users/avneeshchandra/Analyst_tools/avneesh/connect_to_api.ipynb"
# %run /Users/avneeshchandra/Analyst_tools/avneesh/load_dependencies.ipynb
# %run /Users/avneeshchandra/Analyst_tools/avneesh/graphika_trending.ipynb
# # # %run "/Users/thomaslederer/Documents/graphika_scripts/analysis_tools/get_map_dates.ipynb"
# # # %run "/Users/thomaslederer/Documents/graphika_scripts/analysis_tools/connect_to_postgres.ipynb"
# # # %run "/Users/thomaslederer/Documents/graphika_scripts/analysis_tools/get_nodes_data.ipynb"

In [5]:
#This function returns a dataframe with all nodes of a map after fetching their segment data

def get_node_data(map_id, grouped = True):
    """
    This function returns all nodes of a given map with their segment data
    Inputs:
        map_id: Map ID for which to fetch node data
        grouped: Whether or not to merge returned df on group data
    Outputs:
        df0: Dataframe of all nodes in a map with segment data included
    """
    
    #This data should also be saved
    url = "https://api.graphika.com/maps/{}/nodes".format(map_id)
    r = re.get(url,auth=(username, pswd))
    node_data = r.json()

    url = "https://api.graphika.com/maps/{}/clusters".format(map_id)
    r = re.get(url,auth=(username, pswd))
    cluster_data = r.json()


    url = "https://api.graphika.com/maps/{}/groups".format(map_id)
    r = re.get(url,auth=(username, pswd))
    group_data = r.json()

    df_nodes = pd.DataFrame({"screen_name":[n["username"] for n in node_data],\
                            "node_id":[n["node_source_id"] for n in node_data],\
                             "cluster_id":[n["cluster_id"] for n in node_data]})
    df_clusters = pd.DataFrame({"cluster_id":[n["id"] for n in cluster_data], \
                                "cluster_name": [n["name"] for n in cluster_data],\
                                "group_id": [n["group_no"] for n in cluster_data]})
    df_clusters["cluster_id"] = df_clusters["cluster_id"].astype("int")
    df_group = pd.DataFrame({"group_id":[n['group_no'] for n in group_data], \
                            "group_name":[n["name"] for n in group_data]})

    df0 = pd.merge(df_nodes, df_clusters,on = "cluster_id")
    
    if grouped:
    
        df0 = pd.merge(df0,df_group, on = "group_id")
    
        df0 = df0[["screen_name","node_id","cluster_name","group_name","cluster_id"]]
        
        
    df0["node_id"] = df0["node_id"].astype("str")

    return df0


def get_screen_names(map_id):
    """
    This function returns all names of nodes in a given map
    Inputs:
        map_id: Map ID for which to fetch node data
    Outputs:
        df0: Dataframe of all nodes in a map with node names included
    """
    
    url = "https://api.graphika.com/maps/%s/nodes" %map_id
    r = re.get(url,auth=(username, pswd))
    node_hash = r.json()
    screen_names = [x['username'] for x in node_hash]
    service_ids = [x["service_user_id"] for x in node_hash]
    d = {'screen_name':screen_names,'id':service_ids}
    df = pd.DataFrame(d)
    return df

In [6]:
def get_hits_data(feature, map_ids, use_map_dates = True, date_from = False, date_to = False):
    '''This searches within a single map with the option of using the map's creation dates as a param'''
    if use_map_dates:
        date_from, date_to = get_map_dates(map_ids)
    
    if type(map_ids) == int:
        map_id = map_ids
        query = "SELECT * FROM "\
        "(SELECT message_id, hits_twitter_{}.node_id, hit_time, hit_value as hit, map_nodes.map_id, map_nodes.cluster_id FROM hits_twitter_{} \
        join map_nodes on map_nodes.node_id = hits_twitter_{}.node_id) s \
        where s.map_id = {} \
        and s.hit_time BETWEEN '{}'::TIMESTAMP AND '{}'::TIMESTAMP;;".format(feature, feature, feature, map_id, date_from, date_to)
    else:
        
        query = "SELECT * FROM "\
        "(SELECT message_id, hits_twitter_{}.node_id, hit_time, hit_value as hit, map_nodes.map_id, map_nodes.cluster_id FROM hits_twitter_{} \
        join map_nodes on map_nodes.node_id = hits_twitter_{}.node_id) s \
        where s.map_id in ({}) \
        and s.hit_time BETWEEN '{}'::TIMESTAMP AND '{}'::TIMESTAMP;;".format(feature, feature, feature, map_ids, date_from, date_to)
    
    
    
#     print (query)
    print ("getting {} from {} to {}".format(feature, date_from,date_to))
    r = cur.execute(query)
    hits = cur.fetchall()
    hits_df = pd.DataFrame(hits, columns=["message_id","node_id", "time", "hit_value", "map_id", "clu_id"])
    hits_df["node_id"] = hits_df["node_id"].astype("str")
    hits_df["message_id"] = hits_df["message_id"].astype("str")
    hits_df["hit_type"] = feature

    return (hits_df)    

def pull_data_from_s3 (filename):
    '''input the gzip file name'''
    s3 = boto.connect_s3()
    ma_tweets_bucket = s3.get_bucket('ma-tweets')
    results_generator = ma_tweets_bucket.list("manual_searches/{}".format(filename))  
    file = [x for x in results_generator]
    yr_tweets_gz = file[0].get_contents_as_string()
    yr_tweets_json = zlib.decompress(yr_tweets_gz, 16 + zlib.MAX_WBITS)
    data = json.loads(yr_tweets_json)
    
    return (data)

def convert_to_df (tweet_data, write_to_file = False, outfile = "filename"):    
    tweets = []    
    for n in tweet_data:

        row_entry = {}
        row_entry["tweet_id"] = n["id_str"]
        row_entry["user_id"] = n["user"]["id"]
        row_entry["screen_name"] = n["user"]["screen_name"]
        row_entry["time"] = n ['created_at']
        row_entry["text"] = n["text"]

        hashtag_dict = (n["hashtags"])
        hashtag_list = []
        for h in hashtag_dict:
            hashtag_list.append(h["text"]) 
        row_entry["hashtags"] = hashtag_list


        if 'retweeted_status' in n.keys():
            row_entry["retweet"] = True
            row_entry["retweet_screen_name"] = n["retweeted_status"]["user"]["screen_name"]
            row_entry["retweet_tweet_id"] = n["retweeted_status"]["id_str"]

            if "retweet_count" in n["retweeted_status"].keys():
                row_entry["retweet_count"] = n["retweeted_status"]["retweet_count"]

            else: 
                row_entry["retweet_count"] = "Missing"


        else:
            row_entry["retweet"] = False
            row_entry["retweet_screen_name"] = None
            row_entry["retweet_tweet_id"] = None
            row_entry["retweet_count"] = None



        mention_dict = (n["user_mentions"])
        mention_list = []
        for m in mention_dict:
            mention_list.append(m["screen_name"]) 
        row_entry["mentions"] = mention_list



        row_entry["language"] = n['lang']
        
        url_dict = n["urls"]
        url_list = []
        for u in url_dict:
            url_list.append(u['expanded_url'])
        row_entry["urls"] = url_list



        tweets.append(row_entry)  



    tweets_df = pd.DataFrame(tweets)          


    if write_to_file:
        
        writer = pd.ExcelWriter("/Users/avneeshchandra/Downloads/{}.xlsx".format(outfile))
        tweets_df.to_excel(writer)
        writer.save()

    tweets_df["tweet_id"] = tweets_df.tweet_id.astype(str)
    return (tweets_df)

In [7]:
def parse_video_id(url):
    t1 = url.find("watch?v=")
    t2 = url.find("youtu.be/")
    
    if t2>t1:
        sub_0 = url[t2+9:]  
        video_id = sub_0[:12]
    
    elif t1>t2:
        sub_0 = url[t1+8:]  
        video_id = sub_0[:11]
        
    else:
        video_id = ""  
    
    return (video_id)

In [8]:
def get_social_media_links(map_id,from_date = False):

    to_date = False
    map_dates = True
    if from_date != False:
        to_date = datetime.now()
        map_dates = False
        
#     print(to_date,from_date,map_dates)

    print ("getting nodes data...")
    nodes = get_node_data(map_id=map_id)
#     nodes = pd.DataFrame(sb.Nodes(map_id))

    print ("getting urls data...")
    urls = get_hits_data(map_ids=map_id,feature="urls",use_map_dates = map_dates,date_to = to_date, date_from = from_date)

    print ("getting media data...")
    media = get_hits_data(map_ids=map_id,feature="media",use_map_dates = map_dates,date_to = to_date, date_from = from_date)

    df = pd.concat([urls,media])
    df.drop_duplicates("message_id", inplace=True)

    yt_map_data = pd.merge(df,nodes, left_on="node_id", right_on="node_id")

    
    return yt_map_data

In [9]:
def contains_items (urls_df, target_list):

    sm_urls = pd.DataFrame()
    for t in target_list:
        print ("filtering for {}".format(t))
        target_urls = urls_df[urls_df.hit_value.str.contains(t)]
        if not target_urls.empty:
            sm_urls = pd.concat([sm_urls,target_urls])
            
    sm_urls.reset_index (inplace = True)

    return (sm_urls)

In [1]:
#statistics 

#statistics 

def send_youtube_video_request(id_list):
    
    video_data = []
        
#     api_url = "https://www.googleapis.com/youtube/v3/videos?part=snippet&%2CrecordingDetails&"
#     api_url += "id=%s&" % ",".join(id_list)
#     api_url += "key=%s" % youtube_key
    
#     response = re.get(api_url)
    
    SpecificVideoID = "id=%s&" % ",".join(id_list)
    SpecificVideoUrl = 'https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&'+SpecificVideoID+'&key='+youtube_key
#     SpecificVideoUrl = 'https://www.googleapis.com/youtube/v3/channels?part=statistics&id='+SpecificVideoID+'&key='+youtube_key

    response = re.get(SpecificVideoUrl) #makes the call to a specific YouTube
    # videos = json.load(response) #decodes the response so we can work with it
    videos = response.json()
    videoMetadata = [] #declaring our list
#     for video in videos['items']: 
    
    if response.status_code == 200:
        
#         search_results = json.loads(response.content)
#         return videos
#         display(videos)
        for x in videos["items"]:
            video_dict = {}
            video_dict["id"] = (x["id"])
            video_dict["title"] = (x['snippet']["title"])
            video_dict["channel_name"] = (x['snippet']['channelTitle'])
            video_dict['publishedAt'] = x['snippet']['publishedAt']
            try:
                video_dict['description'] = x['snippet']['description']
                video_dict['viewCount'] = x['statistics']['viewCount']
                video_dict['likeCount'] = x['statistics']['likeCount']
                video_dict['dislikeCount'] = x['statistics']['dislikeCount']
                video_dict['favoriteCount'] = x['statistics']['favoriteCount']
                video_dict['commentCount'] = x['statistics']['commentCount']
            except:
                video_dict['description'] = ''
                video_dict['viewCount'] = 0
                video_dict['dislikeCount'] = 0
                video_dict['favoriteCount'] = 0
                video_dict['commentCount'] = 0
            video_data.append(video_dict)
    
    return (video_data)

In [3]:
# def send_youtube_video_request(id_list):
    
#     video_data = []
        
# #     api_url = "https://www.googleapis.com/youtube/v3/videos?part=snippet&%2CrecordingDetails&"
# #     api_url += "id=%s&" % ",".join(id_list)
# #     api_url += "key=%s" % youtube_key
    
# #     response = re.get(api_url)
    
#     SpecificVideoID = "id=%s&" % ",".join(id_list)
#     SpecificVideoUrl = 'https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&'+SpecificVideoID+'&key='+youtube_key
#     response = re.get(SpecificVideoUrl) #makes the call to a specific YouTube
#     # videos = json.load(response) #decodes the response so we can work with it
#     videos = response.json()
#     videoMetadata = [] #declaring our list
# #     for video in videos['items']: 
    
#     if response.status_code == 200:
        
# #         search_results = json.loads(response.content)
# #         return search_results
#         for x in videos["items"]:
#             video_dict = {}
#             video_dict["id"] = (x["id"])
#             video_dict["title"] = (x['snippet']["title"])
#             video_dict["channel_name"] = (x['snippet']['channelTitle'])
#             video_dict['publishedAt'] = x['snippet']['publishedAt']
#             try:
#                 video_dict['description'] = x['snippet']['description']
#                 video_dict['viewCount'] = x['statistics']['viewCount']
#                 video_dict['likeCount'] = x['statistics']['likeCount']
#                 video_dict['dislikeCount'] = x['statistics']['dislikeCount']
#                 video_dict['favoriteCount'] = x['statistics']['favoriteCount']
#                 video_dict['commentCount'] = x['statistics']['commentCount']
#             except:
#                 video_dict['description'] = ''
#                 video_dict['viewCount'] = 0
#                 video_dict['dislikeCount'] = 0
#                 video_dict['favoriteCount'] = 0
#                 video_dict['commentCount'] = 0
#             video_data.append(video_dict)
    
#     return (video_data)

SyntaxError: invalid syntax (<ipython-input-3-e99a72aa07a8>, line 12)

In [4]:
def get_videos_list(yt_links):
    
    
    video_list = list(set(yt_links.u_id))
    video_list = [x for x in video_list if x]
    
    if len(video_list) > 5000:
    
    
        count_vids = pd.DataFrame(yt_links.groupby("u_id").count().iloc[:,1].sort_values(ascending = False))
        video_list = list(count_vids.index[0:5000])
        video_list = [x for x in video_list if x]
    
    return video_list


def get_channels (top_videos):
#     print ("sending {} channels to youtube api".format(len(top_videos)))
    saved_data = []

    counter = 0
    start = 0 
    increase = 10

    no_groups = math.floor(len(top_videos)/increase)



    while counter <= no_groups:

        video_input = top_videos[start:start + increase]
#         print ("sending {} to youtube API".format(video_input))
        video_data = send_youtube_video_request(video_input)

        if video_data:


            saved_data.extend(video_data)
        start += increase
        counter += 1

    video_input = top_videos[start:]
    video_data = send_youtube_video_request(video_input)
    if video_data:
        saved_data.extend(video_data)        
        
    return (saved_data)

In [12]:
def date_filter(df,daysago = 10):
    df.time = pd.to_datetime(df.time)
    end = datetime.now()
    start = datetime.now() - timedelta(days = 10)
#     end = '2020-07-08'
    mask = (df.time > start) & (df.time <= end)
    return df.loc[mask]

In [13]:
def youtube_report(map_id,yt_links = None,num_days = 1):
    
    if yt_links is None:
        print('...Fetching hits')
        urls = pd.DataFrame()
        if type(map_id) == list:
            for an_id in map_id:
                try:
                    urls = urls.append(get_social_media_links(an_id,datetime.now() - timedelta(days = num_days)))
                except:
                    continue
#             urls = urls.drop_duplicates('hit_value')
#             urls = urls.drop_duplicates('message_id')
        else:
            urls = get_social_media_links(map_id,datetime.now() - timedelta(days = num_days))
        yt_links = contains_items(urls,youtube_search)
    #     return yt_links
        yt_links = date_filter(yt_links)
    #     DEBUGGING
    ####
#         display(yt_links)
        yt_links = apply_tags(yt_links.drop(columns = 'cluster_id').rename(columns = {'clu_id':'cluster_id'}))
#         yt_links = yt_links[yt_links.tag.str.contains('Health')]
        
#         return yt_links
    ####
#         yt_links = yt_links[:10000]
    
    u_id_df = pd.DataFrame(list(zip(yt_links.hit_value.tolist(), yt_links.hit_value.apply(parse_video_id).values.tolist())))
#     return yt_links
    yt_links = yt_links.merge(u_id_df,left_on = 'hit_value',right_on = 0).rename(columns = {1:'u_id'}).drop(columns = 0)

#     return yt_links
#     yt_links["u_id"] = yt_links.hit_value.apply(parse_video_id)
#     videos_list = get_videos_list(yt_links[yt_links.tag.str.contains('Health')].drop_duplicates('u_id')[:10000])
    videos_list = get_videos_list(yt_links.drop_duplicates('u_id'))
    chan_details = get_channels(videos_list)
    chan_details = pd.DataFrame(chan_details)
#     return yt_links,chan_details
#     chan_details = chan_details.drop(columns = 'description')
    
    yt_links["video_id"] = True
    yt_links.loc[yt_links.u_id == "","video_id"] = False
    yt_links.loc[yt_links.video_id == False,"u_id"] = yt_links["hit_value"]

    yt_links['date'] = yt_links.time.dt.date
#     return yt_links,chan_details,videos_list
    recent_shares = pd.pivot_table(data = yt_links,values = 'date',index = 'u_id',aggfunc = pd.Series.max)
    recent_shares = recent_shares.rename(columns = {'date':'recent_share_date'})
    
    
    aggby = 'group_name'
#     aggby = 'tag'
#     aggby = 'cluster_name'
    account_shares = pd.pivot_table(data=yt_links, values="node_id", index = "u_id", columns=aggby,aggfunc=pd.Series.nunique,fill_value=0)
    account_shares["total_account_shares"] = account_shares.sum(axis=1)
    account_shares.sort_values("total_account_shares", ascending=False, inplace=True)
    account_shares.reset_index(inplace=True)
    
    account_shares = pd.merge(account_shares,chan_details,left_on="u_id", right_on="id", how="outer")
    account_shares.drop("id",axis=1, inplace=True)


    tweet_shares = pd.pivot_table(data=yt_links, values="message_id", index = "u_id", columns=aggby,aggfunc=pd.Series.nunique,fill_value=0)
    tweet_shares["total_tweet_shares"] = tweet_shares.sum(axis=1)
    tweet_shares.reset_index(inplace=True)
    tweet_shares = tweet_shares[["u_id","total_tweet_shares"]]
#     display(account_shares,tweet_shares)
    counts = pd.merge(account_shares,tweet_shares)
    
    
    urls_detail = yt_links[["u_id","hit_value"]]
    urls_detail.drop_duplicates("u_id", inplace=True)

    counts = pd.merge(counts,urls_detail)
    counts = counts.merge(recent_shares,right_index = True,left_on = 'u_id')
    counts.set_index("u_id",inplace=True)
    
    cols = list(counts.columns)
    cols = ['channel_name','title','hit_value'] + (list(counts.columns))
    cols.remove('channel_name')
    cols.remove("title")
    cols.remove("hit_value")

    counts = counts[cols]
    
    counts = counts.sort_values('total_tweet_shares')

    writer = pd.ExcelWriter("./YT_report_{}_{}.xlsx".format(map_id,str(datetime.now())[:10]))

#     return counts,yt_links
    counts.to_excel(writer,"counts")
#     yt_links[yt_links.tag.str.contains('Health')].drop_duplicates('u_id')[:10000].to_excel(writer,"data")
    yt_links.merge(counts.reset_index()[['u_id']],left_on = 'u_id',right_on = 'u_id')\
    .drop_duplicates('u_id')[:10000].to_excel(writer,"data")

    writer.save()

In [1]:
# youtube_report([2364],num_days = 65)

In [ ]:
# youtube_report(pd.DataFrame(sb.Maps())[pd.DataFrame(sb.Maps()).is_live == True].id.values.tolist(),yt_links = None,num_days = 9)

In [ ]:
def youtube_pinterest_report(map_id,yt_links = None,num_days = 1):
    
    if yt_links is None:
        print('...Fetching hits')
        urls = pd.DataFrame()
        if type(map_id) == list:
            for an_id in map_id:
                urls = urls.append(get_social_media_links(an_id,datetime.now() - timedelta(days = num_days)))
            urls = urls.drop_duplicates('message_id')
        else:
            urls = get_social_media_links(map_id,datetime.now() - timedelta(days = num_days))
        yt_links = contains_items(urls,youtube_search)
        yt_links = date_filter(yt_links)

        yt_links = apply_tags(yt_links.drop(columns = 'cluster_id').rename(columns = {'clu_id':'cluster_id'}))
        display(yt_links,yt_links.tag.value_counts())
#         yt_links = yt_links[yt_links.tag.str.contains('Health')]
    ####
#         yt_links = yt_links[:10000]

    u_id_df = pd.DataFrame(list(zip(yt_links.hit_value.tolist(), yt_links.hit_value.apply(parse_video_id).values.tolist())))
    yt_links = yt_links.merge(u_id_df,left_on = 'hit_value',right_on = 0).rename(columns = {1:'u_id'}).drop(columns = 0)

#     yt_links["u_id"] = yt_links.hit_value.apply(parse_video_id)
    videos_list = get_videos_list(yt_links[yt_links.tag.str.contains('Health')].drop_duplicates('u_id')[:10000])
#     videos_list = get_videos_list(yt_links.drop_duplicates('u_id'))
    chan_details = get_channels(videos_list)
    chan_details = pd.DataFrame(chan_details)
    chan_details = chan_details.drop(columns = 'description')
    
    yt_links["video_id"] = True
    yt_links.loc[yt_links.u_id == "","video_id"] = False
    yt_links.loc[yt_links.video_id == False,"u_id"] = yt_links["hit_value"]

    yt_links['date'] = yt_links.time.dt.date
#     return yt_links,chan_details,videos_list
    recent_shares = pd.pivot_table(data = yt_links,values = 'date',index = 'u_id',aggfunc = pd.Series.max)
    recent_shares = recent_shares.rename(columns = {'date':'recent_share_date'})
    
    account_shares = pd.pivot_table(data=yt_links, values="node_id", index = "u_id", columns="tag",aggfunc=pd.Series.nunique,fill_value=0)
    account_shares["total_account_shares"] = account_shares.sum(axis=1)
    account_shares.sort_values("total_account_shares", ascending=False, inplace=True)
    account_shares.reset_index(inplace=True)
    
    account_shares = pd.merge(account_shares,chan_details,left_on="u_id", right_on="id", how="outer")
    account_shares.drop("id",axis=1, inplace=True)


    tweet_shares = pd.pivot_table(data=yt_links, values="message_id", index = "u_id", columns="tag",aggfunc=pd.Series.nunique,fill_value=0)
    tweet_shares["total_tweet_shares"] = tweet_shares.sum(axis=1)
    tweet_shares.reset_index(inplace=True)
    tweet_shares = tweet_shares[["u_id","total_tweet_shares"]]
#     display(account_shares,tweet_shares)
    counts = pd.merge(account_shares,tweet_shares)
    
    
    urls_detail = yt_links[["u_id","hit_value"]]
    urls_detail.drop_duplicates("u_id", inplace=True)

    counts = pd.merge(counts,urls_detail)
    counts = counts.merge(recent_shares,right_index = True,left_on = 'u_id')
    counts.set_index("u_id",inplace=True)
    
    cols = list(counts.columns)
    cols = ['channel_name','title','hit_value'] + (list(counts.columns))
    cols.remove('channel_name')
    cols.remove("title")
    cols.remove("hit_value")

    counts = counts[cols]
    
    counts = counts.sort_values('recent_share_date')

    writer = pd.ExcelWriter("./YT_report_{}.xlsx".format(map_id))

#     return counts,yt_links
    counts.to_excel(writer,"counts")
    yt_links[yt_links.tag.str.contains('Health')].drop_duplicates('u_id')[:10000].to_excel(writer,"data")
#     yt_links.to_excel(writer,"data")

    writer.save()
    return counts,yt_links

In [ ]:
# c_temp,y_temp = youtube_pinterest_report([2289,2434,2364],num_days = 30)

In [ ]:
# c_temp,y_temp = youtube_pinterest_report([2719],num_days = 30)

In [ ]:
def tag_lookup(rcvd_series):
    """
    This function returns a Series of tags
    Inputs:
        rcvd_series: An input of cluster IDs
    Outputs:
        load_tags().loc[map_id,cluster_id]['TAG']: Tag name
    """
    global all_tags
    map_id = int(rcvd_series['map_id'])
    cluster_id = int(rcvd_series['cluster_no'])
#     display(map_id,cluster_id)
#     unique_id = str(map_id) + '.' + str(cluster_id)
#     display(rcvd_series)
    try:
        return all_tags.loc[map_id,cluster_id]['tag']
    except:
        return 'Unknown'

In [ ]:
def reddit_report(map_id,reddit_links = None,num_days = 1):
    
    if type(reddit_links) == None:
        urls = pd.DataFrame()
        if type(map_id) == list:
            for an_id in map_id:
                urls = urls.append(get_social_media_links(an_id,datetime.now() - timedelta(days = num_days)))
            urls = urls.drop_duplicates('hit_value')
        else:
            urls = get_social_media_links(map_id,datetime.now() - timedelta(days = num_days))
        reddit_links = contains_items(urls,reddit_search)
    #     return yt_links
        reddit_links = date_filter(reddit_links)
    #     DEBUGGING
        reddit_links = reddit_links[:10000]

#     pin_links = contains_items(urls,pinterest_search)
        

    reddit_links["reddit_name"] = parse_subreddits(reddit_links)['name']
    reddit_links["reddit_type"] = parse_subreddits(reddit_links)['type']

    reddit_links['date'] = reddit_links.time.dt.date
    recent_shares = pd.pivot_table(data = reddit_links,values = 'date',index = 'reddit_name',aggfunc = pd.Series.max)
    recent_shares = recent_shares.rename(columns = {'date':'recent_share_date'})
    
    account_shares = pd.pivot_table(data=reddit_links, values="node_id", index = "reddit_name", columns="group_name",aggfunc=pd.Series.nunique,fill_value=0)
    account_shares["total_account_shares"] = account_shares.sum(axis=1)
    account_shares.sort_values("total_account_shares", ascending=False, inplace=True)
    account_shares.reset_index(inplace=True)

    tweet_shares = pd.pivot_table(data=reddit_links, values="message_id", index = "reddit_name", columns="group_name",aggfunc=pd.Series.nunique,fill_value=0)
    tweet_shares["total_tweet_shares"] = tweet_shares.sum(axis=1)
    tweet_shares.reset_index(inplace=True)
    tweet_shares = tweet_shares[["reddit_name","total_tweet_shares"]]
#     display(account_shares,tweet_shares)
    counts = pd.merge(account_shares,tweet_shares)
    
    
    urls_detail = reddit_links[["reddit_name","reddit_type","hit_value"]]
    urls_detail.drop_duplicates("reddit_name", inplace=True)

    counts = pd.merge(counts,urls_detail)
    counts = counts.merge(recent_shares,right_index = True,left_on = 'reddit_name')
    counts.set_index("reddit_name",inplace=True)
    
    cols = list(counts.columns)
    cols = ['reddit_name','reddit_type','hit_value'] + (list(counts.columns))
    cols.remove('reddit_name')
    cols.remove('reddit_type')
    cols.remove("hit_value")

    counts = counts[cols]
    
    counts = counts.sort_values('recent_share_date')

    writer = pd.ExcelWriter("./reddit_report_{}.xlsx".format(map_id))

    return counts,reddit_links

    counts.to_excel(writer,"counts")
    reddit_links.to_excel(writer,"data")

    writer.save()
    

In [ ]:
def pinterest_report(map_id,pt_links = None,num_days = 1):
    
    if type(pt_links) == None:
        urls = pd.DataFrame()
        if type(map_id) == list:
            for an_id in map_id:
                urls = urls.append(get_social_media_links(an_id,datetime.now() - timedelta(days = num_days)))
            urls = urls.drop_duplicates('hit_value')
        else:
            urls = get_social_media_links(map_id,datetime.now() - timedelta(days = num_days))
        pt_links = contains_items(urls,pinterest_search)
    #     return yt_links
        pt_links = date_filter(pt_links)
    #     DEBUGGING
        pt_links = pt_links[:10000]

#     pin_links = contains_items(urls,pinterest_search)
        

#     pt_links["pinterest_name"] = parse_subreddits(reddit_links)['name']
#     pt_links["reddit_type"] = parse_subreddits(reddit_links)['type']

    pt_links['date'] = pt_links.time.dt.date
    recent_shares = pd.pivot_table(data = pt_links,values = 'date',index = 'hit_value',aggfunc = pd.Series.max)
    recent_shares = recent_shares.rename(columns = {'date':'recent_share_date'})
    
    account_shares = pd.pivot_table(data=pt_links, values="node_id", index = "hit_value", columns="group_name",aggfunc=pd.Series.nunique,fill_value=0)
    account_shares["total_account_shares"] = account_shares.sum(axis=1)
    account_shares.sort_values("total_account_shares", ascending=False, inplace=True)
    account_shares.reset_index(inplace=True)

    tweet_shares = pd.pivot_table(data=pt_links, values="message_id", index = "hit_value", columns="group_name",aggfunc=pd.Series.nunique,fill_value=0)
    tweet_shares["total_tweet_shares"] = tweet_shares.sum(axis=1)
    tweet_shares.reset_index(inplace=True)
    tweet_shares = tweet_shares[["hit_value","total_tweet_shares"]]
#     display(account_shares,tweet_shares)
    counts = pd.merge(account_shares,tweet_shares)
    
    
    urls_detail = pt_links[["hit_value"]]
    urls_detail.drop_duplicates("hit_value", inplace=True)

    counts = pd.merge(counts,urls_detail)
    counts = counts.merge(recent_shares,right_index = True,left_on = 'hit_value')
    counts.set_index("hit_value",inplace=True)
    
    cols = list(counts.columns)
    cols = ['hit_value'] + (list(counts.columns))
#     cols.remove('reddit_name')
#     cols.remove('reddit_type')
    cols.remove("hit_value")

    counts = counts[cols]
    
    counts = counts.sort_values('recent_share_date')

    return counts,pt_links

    writer = pd.ExcelWriter("./pinterest_report_{}.xlsx".format(map_id))

    counts.to_excel(writer,"counts")
    pt_links.to_excel(writer,"data")

    writer.save()
    

In [ ]:
def domain_reports(map_id,num_days = 1):
    urls = pd.DataFrame()
    if type(map_id) == list:
        for an_id in map_id:
            urls = urls.append(get_social_media_links(an_id,datetime.now() - timedelta(days = num_days)))
        urls = urls.drop_duplicates('hit_value')
    else:
        urls = get_social_media_links(map_id,datetime.now() - timedelta(days = num_days))
    pin_links = contains_items(urls,pinterest_search)
    reddit_links = contains_items(urls,reddit_search)
    youtube_links = contains_items(urls,youtube_search)
    
    writer = pd.ExcelWriter("./domains_report_{}.xlsx".format(map_id))
    
    calls = {'pinterest':[pinterest_report,pin_links],'reddit':[reddit_report,reddit_links],'youtube':[youtube_report,youtube_links]}
    
    for x in ['pinterest','reddit','youtube']:
        one,two = calls[x][0](map_id,calls[x][1])
        one.to_excel(writer,x + '_counts')
        two.to_excel(writer,x + '_data')

    writer.save()

In [ ]:
# domain_reports(2364,7)

In [ ]:
def parse_subreddits(df):
    link_titles = []
    for r in df.hit_value.tolist():
        try:
            link_titles.append(r.split('/')[1:3])
#         print(r.split('/')[1],r)
            continue
        except:
            link_titles.append('Unknown')
            continue
        link_titles.append('False positive')
    return pd.DataFrame(link_titles,columns = ['type','name'])

In [ ]:
#merge_date_cells

# import xlsxwriter
# import string

# cell_markers = []
# start_cell = 2
# end_cell = 0
# for d in temp.recent_share_date.astype(str).unique().tolist():
#     end_cell = start_cell + temp.recent_share_date.value_counts().loc[d].values[0] - 1
#     print('starting at {} and ending at {}'.format(start_cell,end_cell))
#     cell_markers.append([start_cell,end_cell])
#     start_cell = end_cell + 1
    
# col_letter = string.ascii_uppercase[temp.columns.tolist().index('recent_share_date')]
# cols_to_merge = []
# for x in cell_markers:
#     cols_to_merge.append([col_letter + str(x[0]),col_letter + str(x[1])])

# worksheet = workbook['counts']
# for x in cols_to_merge:
# #     print('{}:{}'.format(x[0],x[1]))
#     worksheet.merge_cells('{}:{}'.format(x[0],x[1]))
# workbook.save('/Users/avneeshchandra/Downloads/trial.xlsx')

In [ ]:
def get_list_counts (df, feature, count_by_value, map_id = False):
        
  
    s = df.loc[df[feature].str.len() > 0,[feature, count_by_value]]

    s1 = s[feature].apply(pd.Series)\
        .merge(s, left_index = True, right_index = True)\
        .drop([feature], axis = 1)\
        .melt(id_vars = [count_by_value], value_name = feature)\
        .drop("variable", axis = 1) \
        .dropna()
    
    
    if map_id:
        map_nodes = get_node_data(map_id=map_id)
        df.user_id = df.user_id.astype(str)
        map_tweets = pd.merge(nodes,df, left_on="node_id",right_on='user_id')
    
        s1.user_id = s1.user_id.astype(str)

        data = pd.merge(s1,map_nodes,left_on="user_id",right_on="node_id")

        counts_df = count_by_group(data,feature)

    else:
        
        counts = s1.groupby(feature)[count_by_value].nunique().sort_values(ascending = False)
        counts_df = pd.DataFrame(counts)
        
    counts_df = counts_df[(counts_df.index.str.contains("youtube"))|(counts_df.index.str.contains("youtu.be"))]
    return counts_df 
    
 
        
def initial_data_report (filename, filter_by_map = False):
    data = pull_data_from_s3(filename)
    df = convert_to_df(data) 
    
    if filter_by_map:
        counts = get_list_counts(df,"urls",'user_id', map_id = filter_by_map)
        writer = pd.ExcelWriter("/Users/thomaslederer/Downloads/ytlinks_initial{}.xlsx".format(filter_by_map))

       
    else:
        urls_tweet_counts = get_list_counts(df,"urls",'tweet_id')
        urls_user_counts = get_list_counts(df,"urls",'user_id')
        counts = pd.merge(urls_tweet_counts,urls_user_counts,left_index=True, right_index=True)
        
        d1 = date.today().strftime("%Y.%m.%d")
        writer = pd.ExcelWriter("/Users/thomaslederer/Downloads/ytlinks_initial{}.xlsx".format(d1))
    
    counts.reset_index(inplace=True)
    counts["u_id"] = counts.urls.apply(parse_video_id)
    channels = get_channels(list(counts["u_id"]))
    channels_df = pd.DataFrame(channels)
    channels_df.drop_duplicates(inplace=True)

    final_df = pd.merge(channels_df,counts, left_on="id",right_on="u_id", how="right")
    final_df.sort_values("total", inplace = True, ascending=False)
    
    
    final_df.to_excel(writer)

    writer.save()

In [ ]:
def count_by_group (data, feature):

    df = pd.pivot_table(data = data,
                index = feature,
                columns = "group_name",
                aggfunc = pd.Series.nunique,
                values = "user_id",
                fill_value = 0)

    df["total"] = df.sum(axis = 1)
    df.sort_values(by = "total",inplace=True, ascending=False)
    
    return (df)

                                
        

In [ ]:
def get_live_maps (write_to_file = False):
    url = "https://api-production.graphika.io/admin/clustermaps/live"
    r = re.get(url,auth=(username, pswd))
    map_data = r.json()
    
    entries = []
    for e in map_data["data"]:
        entries.append(e)   
    df = pd.DataFrame(entries)
    df.drop("capture_live_data", axis=1, inplace = True)
    
    
    now = datetime.now()
    date =  str(now)
    date = date[:date.find(" ")]
    
    
    if write_to_file:
        filename = "/Users/thomaslederer/Downloads/%s_live_maps.xlsx" %date
        writer = pd.ExcelWriter(filename)
        df.to_excel(writer, index=False)
        writer.save()
    
    return (df)


def get_hits_groupby(feature,date_from, date_to,map_ids,groupby="clusters",):
    
    
    #default is to not specify map id and get all live maps
    #if not map_ids:
    #    live_maps = get_live_maps()
    #    map_ids = tuple(live_maps.id.astype(str))
#     if len(map_ids) == 1:
#         map_ids = '(' + str(map_ids[0]) + ')'

    if groupby=="maps":
        cols = ["hit","map_id","count"]
        query = "Select s.hits, s.map_id, count(distinct s.node_id) \
from (select hits_twitter_{}.node_id,hit_value as hits,hit_time,map_id from hits_twitter_{} join map_nodes on map_nodes.node_id = hits_twitter_{}.node_id) \
s  where s.hit_time between '{}' AND '{}' and s.map_id in {} \
group by s.hits, map_id \
order by count(distinct s.node_id) desc;".format(feature, feature, feature, date_from, date_to,'(' + str(map_ids[0]) + ')')
    
    if groupby=="clusters":
        cols = ["hit","clu_id","map_id","count"] 
        query = "Select s.hits, s.cluster_id,s.map_id, count(distinct s.node_id) \
from (select hits_twitter_{}.node_id,hit_value as hits,hit_time,map_id,cluster_id from hits_twitter_{} join map_nodes on map_nodes.node_id = hits_twitter_{}.node_id) \
s  where s.hit_time between '{}' AND '{}' and s.map_id in {} \
group by s.hits, cluster_id, map_id \
order by count(distinct s.node_id) desc;".format(feature, feature, feature, date_from, date_to,'(' + str(map_ids[0]) + ')')
    
    
    #print (query)
    r = cur.execute(query)
    hits = cur.fetchall()
    hits_df = pd.DataFrame(hits, columns = cols) 
    return (hits_df)

In [ ]:
def supplement_map_name (df):
    r = re.get("https://api.graphika.com/maps",auth=(username, pswd))
    n_hash = r.json()
    map_names = pd.DataFrame(n_hash)
    
    return_df = pd.merge(df,map_names,right_on="id",left_on="map_id")
    return_df.drop(["id","map_id"], axis =1, inplace = True)
    
    return (return_df)


def live_report (date_from, date_to):
    
#     writer = pd.ExcelWriter(f"/Users/thomaslederer/Downloads/live_map_yt_links_{date_from}{date_to}.xlsx")
    writer = pd.ExcelWriter(f"/Users/avneeshchandra/Downloads/live_map_yt_links_2643_{date_from}{date_to}.xlsx")
    
#     live_maps = get_live_maps()
#     live_maps_tuple = tuple(live_maps.id)
    live_maps_tuple = tuple([2643])
    
    urls_groupby = get_hits_groupby("urls",date_from,date_to, map_ids=live_maps_tuple)
    media_groupby = get_hits_groupby("media",date_from,date_to, map_ids=live_maps_tuple)
#     print (urls_groupby)
    urls_groupby = urls_groupby[(urls_groupby.hit.str.contains("youtube.com"))|(urls_groupby.hit.str.contains("youtu.be"))]
    media_groupby = media_groupby[(media_groupby.hit.str.contains("youtube.com"))|(media_groupby.hit.str.contains("youtu.be"))]
    
    media_groupby["type"] = "media"
    urls_groupby["type"] = "urls"
    
    yt_hits_groupby = pd.concat([media_groupby,urls_groupby])
    yt_hits_groupby = supplement_map_name(yt_hits_groupby)
#     yt_hits_groupby['name'] = ['mapname' for x in range(yt_hits_groupby.shape[0])]
    
#     return yt_hits_groupby
    yt_pivot = yt_hits_groupby.pivot_table(index="hit", columns="name").fillna(0)

    
    
    media = get_hits_data("media",live_maps_tuple, use_map_dates = False, date_from=date_from, date_to=date_to)
    media = media[(media.hit_value.str.contains("youtube.com"))|(media.hit_value.str.contains("youtu.be"))]

    urls = get_hits_data("urls",live_maps_tuple, use_map_dates = False, date_from=date_from, date_to=date_to)
    urls = urls[(urls.hit_value.str.contains("youtube.com"))|(urls.hit_value.str.contains("youtu.be"))]
    
    yt_hits = pd.concat([media,urls])

    account_totals = pd.DataFrame(yt_hits.groupby("hit_value").nunique("node_id").iloc[:,1].sort_values(ascending = False))
    total_shares =  pd.DataFrame(yt_hits.groupby("hit_value").count()["message_id"].sort_values(ascending = False))                                    
    totals = pd.merge(account_totals,total_shares,left_index=True, right_index=True)
    totals.columns = ["count_total_users", "count_total_shares"]

    df = pd.merge(totals,yt_pivot,right_index=True, left_index=True)
    
    df.reset_index(inplace=True)
    df["u_id"] = df["index"].apply(parse_video_id)
    channels = get_channels(list(df["u_id"][:5000]))
    channels_df = pd.DataFrame(channels)
    channels_df.drop_duplicates(inplace=True)
#     return [df,channels_df]

    final_df = pd.merge(channels_df,df, left_on="id",right_on="u_id", how="right")
    #final_df.set_axis("index")
    #final_df.index.names = ['url']
    #df = df.rename(columns={'node_id': 'unique_accounts', 'message_id': 'total_shares'})



    final_df.to_excel(writer)
    writer.save()
    
    return final_df